In [16]:
import cv2
import numpy as np

In [17]:
def comic(img):
    # do edge detection on a grayscale image
    gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
    edges = cv2.blur(gray, (3, 3)) # this blur gets rid of some noise
    edges = cv2.Canny(edges, 50, 150, apertureSize=3) # this is the edge detection
    cv2.imshow('canny', edges)
    # the edges are a bit thin, this blur and threshold make them a bit fatter
    kernel = np.ones((3,3), dtype=np.float) / 12.0
    edges = cv2.filter2D(edges, 0, kernel)
    edges = cv2.threshold(edges, 50, 255, 0)[1]
    cv2.imshow('threshlod', edges)
    # and back to colour...
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    cv2.imshow('colored edge', edges)
    # this is the expensive operation, it blurs things but keeps track of
    # colour boundaries or something, heck, just play with it
    shifted = cv2.pyrMeanShiftFiltering(img, 5, 20)
    cv2.imshow('shifted', shifted)
    # now compose with the edges, the edges are white so take them away
    # to leave black
    return cv2.subtract(shifted, edges)

In [18]:
def render(img_rgb):
    numDownSamples = 2 # number of downscaling steps
    numBilateralFilters = 7  # number of bilateral filtering steps

    # -- STEP 1 --
    # downsample image using Gaussian pyramid
    img_color = img_rgb
    for i in range(numDownSamples):
        img_color = cv2.pyrDown(img_color)

    # repeatedly apply small bilateral filter instead of applying
    # one large filter
    for i in range(numBilateralFilters):
        img_color = cv2.bilateralFilter(img_color, 9, 9, 7)

    # upsample image to original size
    for i in range(numDownSamples):
        img_color = cv2.pyrUp(img_color)

    # -- STEPS 2 and 3 --
    # convert to grayscale and apply median blur
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_blur = cv2.medianBlur(img_gray, 7)

    # -- STEP 4 --
    # detect and enhance edges
    img_edge = cv2.adaptiveThreshold(img_blur, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)

    # -- STEP 5 --
    # convert back to color so that it can be bit-ANDed
    # with color image
    img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2RGB)
    return cv2.bitwise_and(img_color, img_edge)

In [19]:
img = cv2.imread('../../../car_dataset/AC_Schnitzer-GP3_10_Concept_10.jpg', cv2.IMREAD_COLOR)
# for i in range(5):
#     blur = cv2.bilateralFilter(img, 9, 9, 7, cv2.BORDER_TRANSPARENT)
# resize = cv2.resize(blur, dsize=(256, 256), interpolation=cv2.INTER_LINEAR_EXACT)
img_color = img
for i in range(2):
    img_color = cv2.pyrDown(img_color)
    print(img_color.shape, img.shape)
num_iter = 7
for i in range(num_iter):
    img_color = cv2.bilateralFilter(img_color, d=9, sigmaColor=9, sigmaSpace=7)
for i in range(2):
    img_color = cv2.pyrUp(img_color)
    print(img_color.shape, img.shape)

img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
img_blur = cv2.medianBlur(img_gray, 7)
img_edge = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5 , 2)
cv2.imshow('img_edge', img_edge)
img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2RGB)

cartoon = cv2.bitwise_and(img_color, img_edge)

comic1 = comic(img)

cv2.imshow('img', img)
# cv2.imshow('blur', blur)
cv2.imshow('img_color', img_color)
cv2.imshow('img_edge', img_edge)
cv2.imshow('render', render(img))
cv2.imshow('cartoon', cartoon)
cv2.imshow('comic', comic1)
# cv2.imshow('resize', resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

(600, 800, 3) (1200, 1600, 3)
(300, 400, 3) (1200, 1600, 3)
(600, 800, 3) (1200, 1600, 3)
(1200, 1600, 3) (1200, 1600, 3)


In [12]:
# Load two images
img1 = cv2.imread('2502287818_41e4b0c4fb_z.jpg')
img2 = cv2.imread('logo.jpg')
# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols]
# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
cv2.imshow('threshold ', mask)

mask_inv = cv2.bitwise_not(mask)
cv2.imshow('mask_inv', mask_inv)

# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
cv2.imshow('bitwise_and1', img1_bg)

# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2,img2,mask = mask)
cv2.imshow('bitwise_and2', img2_fg)

# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
cv2.imshow('dst', dst)
img1[0:rows, 0:cols ] = dst
print(img1[0:rows, 0:cols ].shape)
print(img1[:rows, :cols].shape)
cv2.imshow('res',img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

(186, 149, 3)
(186, 149, 3)


In [ ]:
merged_image=np.zeros((1200, 3200, 3))
print(merged_image.shape)

print(img2[:186, :149])
print("---------------------")
print(img2[:, :149])

In [ ]:
x = np.arange(20)
print(x)

In [ ]:
x.shape

In [ ]:
x[0:10]

In [ ]:
x[:10]

In [ ]:
x[10:]

In [ ]:
x[:10]

In [15]:
vis = np.concatenate((img1, img2), axis=0)
cv2.imshow('vis', vis)
vis2 = np.concatenate((img1, img2), axis=0)
cv2.imshow('vis', vis2)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 425 and the array at index 1 has size 149

In [ ]:
room2[np.where(logo_mask == 255)] = logo[np.where(logo_mask == 255)]
